In [17]:
import tensorflow as tf
from tensorflow.contrib import slim

import numpy as np

import time
import os

from PIL import Image
from matplotlib import gridspec
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
log_dir = '/Users/nikki/Desktop/models/research/deeplab/graph/1/'
model_dir_path = '/Users/nikki/Development/deeplab-img-segmentation/model_graph/deeplabv3_pascal_train_aug/'

LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

In [3]:
def create_color_map(num_labels):
    colormap = np.zeros((num_labels, 3), dtype=np.uint8)
    for i in range(num_labels):
        colormap[i, :] = i * 10
    return colormap

In [4]:
def visualize_seg(image, seg_map):
    color_map = create_color_map(len(LABEL_NAMES))
    
    plt.figure(figsize=(20, 5))

    plt.subplot(141)
    plt.imshow(image)
    plt.axis('off')
    
    plt.subplot(142)
    plt.imshow(image)
    plt.imshow(color_map[seg_map], alpha=0.8)
    plt.axis('off')
    
    plt.subplot(143)
    plt.imshow(color_map[seg_map])
    plt.axis('off')
    
    unique_labels = np.unique(seg_map)
    plt.subplot(144)
    plt.imshow([color_map[unique_labels]], interpolation='nearest')
    plt.xticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.grid('off')

In [5]:
def resize_img(image):
    INPUT_SIZE = 513
    
    width, height = image.size
    resize_ratio = 1.0 * INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    
    return resized_image

In [6]:
def pad_to_bounding_box(sess, image):
    img_placeholder = tf.placeholder(shape=(None, None, 3), dtype=tf.float32)
    
    rrr = tf.constant(shape=[1], dtype=tf.float32, value=127.5)
    qq = img_placeholder - rrr
    
    resized_img = tf.image.pad_to_bounding_box(image=qq, 
                                               offset_height=0, 
                                               offset_width=0, 
                                               target_height=513,
                                              target_width=513)
    
    rezzz = resized_img + rrr
    
    res_img = sess.run(rezzz, feed_dict={img_placeholder: image})
    return res_img

In [13]:
class DeepLab(object):
    OUTPUT_TENSOR_NAME_PAD = 'Pad:0'
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    
    
    INPUT_TENSOR_NAME_ADD = 'add_2:0'
    OUTPUT_TENSOR_NAME_TEST = 'SemanticPredictions/size:0'
    
    DECODER_OUTPUT_TENSOR = 'decoder/decoder_conv1_pointwise/Relu:0'
    
    def __init__(self):
        self.graph = tf.Graph()

        file = open(model_dir_path + 'frozen_inference_graph.pb', mode='rb')
        graph_def = tf.GraphDef.FromString(file.read())
        file.close()

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.Session(graph=self.graph)


    def run(self, image):
        t0 = time.time()
        
        resized_image = resize_img(image=image)
        width, height = resized_image.size
        
        with tf.Session() as sess:
            bounded_image = pad_to_bounding_box(sess, resized_image)
        
        # 1x129x129x256
        decoder_output = self.sess.run(
            self.DECODER_OUTPUT_TENSOR,
            feed_dict={
                       self.INPUT_TENSOR_NAME_ADD: bounded_image,
                       self.OUTPUT_TENSOR_NAME_TEST: np.array([1, height, width])
                    })
        
#         with tf.Session() as sess:
#             # 1x129x129x21
# #             logits = slim.conv2d(decoder_output, num_outputs=21, kernel_size=1)
            
#             upsampled_logits = tf.image.resize_bilinear(images=decoder_output, size=(513, 513), align_corners=True)
#             prediction = tf.argmax(upsampled_logits, axis=3)
#             seg_map_tensor = tf.slice(input_=prediction, begin=(0, 0, 0), size=[1, height, width])
            
            
#             sess.run(tf.global_variables_initializer())
#             seg_map = sess.run(seg_map_tensor)[0]
            
        print(time.time() - t0)
        return decoder_output
#         return resized_image, seg_map
        
    
    def save_graph(self):
        writer = tf.summary.FileWriter(logdir=log_dir+'my_net', graph=self.graph)

In [14]:
deeplab = DeepLab()

In [15]:
image = Image.open('/Users/nikki/Development/deeplab-img-segmentation/data/train_color_batch/170908_065647862_Camera_6.jpg')
decoder_out = deeplab.run(image)

16.90623688697815


In [16]:
print(decoder_out.shape)

(1, 129, 129, 256)


In [20]:
def save_decoder_outputs(folder):
    img_names = os.listdir(folder)
    
    decoder_outputs = np.array([deeplab.run(Image.open(folder+img)) for img in img_names])
    decoder_outputs.dump('train_color_batch.npy')
    
    print(decoder_outputs.shape)

In [21]:
save_decoder_outputs('data/train_color_batch/')

13.107985973358154
13.463176012039185
12.997730016708374
12.814552068710327
12.771435022354126
12.60319209098816
12.637753963470459
12.608979940414429
12.719921112060547
(9, 1, 129, 129, 256)
